<a href="https://colab.research.google.com/github/acmilannesta/Bert-embedding/blob/master/Xlnet_adultreadmission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://storage.googleapis.com/xlnet/released_models/cased_L-12_H-768_A-12.zip
!unzip cased_L-12_H-768_A-12.zip

In [0]:
!pip install keras_xlnet

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
from keras_xlnet import Tokenizer, load_trained_model_from_checkpoint, ATTENTION_TYPE_BI

checkpoint_path = 'xlnet_cased_L-12_H-768_A-12'

tokenizer = Tokenizer(os.path.join(checkpoint_path, 'spiece.model'))
xlnet_model = load_trained_model_from_checkpoint(
    config_path=os.path.join(checkpoint_path, 'xlnet_config.json'),
    checkpoint_path=os.path.join(checkpoint_path, 'xlnet_model.ckpt'),
    batch_size=16,
    memory_len=512,
    target_len=128,
    in_train_phase=False,
    attention_type=ATTENTION_TYPE_BI,
)


In [91]:
tokenizer.SYM_CLS

3

In [0]:
maxlen=300
def seq_padding(X, padding=tokenizer.SYM_PAD):
  L = [len(x) for x in X]
  ML = max(L)
  return np.array([np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X])

class data_generator:
  def __init__(self, data, batch_size=8, memlen=512):
    self.data = data
    self.batch_size = batch_size
    self.memlen = memlen
    self.steps = len(self.data) // self.batch_size
    if len(self.data) % self.batch_size != 0:
        self.steps += 1
  def __len__(self):
    return self.steps
  def __iter__(self):
    while True:
      idxs = list(range(len(self.data)))
      np.random.shuffle(idxs)
      X1, X2, X3, Y = [], [], [], []
      for i in idxs:
        d = self.data[i]
        text = d[0][:maxlen]
        x1 = tokenizer.encode(text=text)
        x2 = [float(tokenizer.SYM_CLS) for i in range(len(x1))]
        x3 = [float(self.memlen) for i in range(len(x1))]
        y = d[1]
        X1.append(x1)
        X2.append(x2)
        X3.append(x3)
        Y.append([y])
        if len(X1) == self.batch_size or i == idxs[-1]:
          X1 = seq_padding(X1)
          X2 = seq_padding(X2)
          X3 = seq_padding(X3, padding=self.memlen)
          Y = seq_padding(Y)
          yield [X1, X2, X3], Y
          X1, X2, X3, Y = [], [], [], []

In [0]:
import json
import numpy as np
import pandas as pd
from random import choice
from keras_bert import load_trained_model_from_checkpoint, Tokenizer
import re, os, gc
import codecs
from keras.layers import *
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K
from keras.optimizers import Adam
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score


In [0]:
data = pd.read_csv('drive/My Drive/Adult readmission/Combined_vars_04092019_notes.csv')
data = data[['opnote_1', 'CODE_REHOSP']].dropna()
data.CODE_REHOSP = np.where(data.CODE_REHOSP==2, 0, 1)
random_order = list(range(len(data)))
np.random.shuffle(random_order)
train_data = [data.iloc[j] for i, j in enumerate(random_order) if i % 5 != 0]
valid_data = [data.iloc[j] for i, j in enumerate(random_order) if i % 5 == 0]

In [0]:
x1_in = Input(shape=(None,))
x2_in = Input(shape=(None,))
x3_in = Input(shape=(None,))

x = xlnet_model([x1_in, x2_in, x3_in])
x = Lambda(lambda x: x[:, 0])(x)
p = Dense(1, activation='sigmoid')(x)

model = Model([x1_in, x2_in, x3_in], p)
# def auroc(y_true, y_pred):
#     return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(1e-5),
    metrics=['acc']
)


In [94]:
train_D = data_generator(train_data)
valid_D = data_generator(valid_data)
model.fit_generator(
    train_D.__iter__(),
    steps_per_epoch=len(train_D),
    epochs=2,
    validation_data=valid_D.__iter__(),
    validation_steps=len(valid_D)
)

Epoch 1/2
147/147 [==============================] - 149s 1s/step - loss: 0.6632 - acc: 0.6611 - val_loss: 0.5903 - val_acc: 0.7245
Epoch 2/2
147/147 [==============================] - 111s 758ms/step - loss: 0.6150 - acc: 0.6900 - val_loss: 0.6177 - val_acc: 0.7245


In [0]:
pred = model.predict_generator(valid_D.__iter__(), len(valid_D))

In [96]:
print('Accuracy Score: {:.2f}%'.format(accuracy_score([x[1] for x in valid_data], [int(x>0.5) for x in pred])*100))
print('ROC-AUC: {:.2f}%'.format(roc_auc_score([x[1] for x in valid_data], pred)*100))

Accuracy Score: 72.45%
ROC-AUC: 46.24%
